### 仮説

「平均年収で大きな格差がある港区と足立区において、アルバイト求人の時給にも同様の格差が存在する。港区の求人の方が、足立区の求人よりも平均時給が高い。」

東京23区内の平均年収データによると、港区が最も高く、足立区が最も低いことが判明している。この地域間の経済格差は、アルバイトやパート求人の時給にも反映されているのではないかと考えた。
この仮説の検証により、地域の経済状況が非正規雇用の賃金水準にどの程度影響を与えているかを明らかにすることができる。また、同一労働市場である東京都内でも、地域による賃金格差が存在するかを確認することができる。

### スクレイピングのコード解説

1. データベース設計と作成


* SQLiteを採用し、単一ファイルでデータを管理
* 自動増分のID、会社名、時給、地区名、タイムスタンプを持つテーブル構造
* CREATE TABLE IF NOT EXISTS で重複作成を防止


2. スクレイピング前の準備


* 正規表現パターンを事前にコンパイルして処理を効率化
* データベース接続を確立
* ページ数の上限を設定（デフォルト50ページ）


3. ページ取得とエラー処理


* URLを動的に生成してページごとにアクセス
* 3秒のインターバルでサーバー負荷を分散
* ステータスコード200以外は処理を中断


4. データ抽出の工夫


* HTML構造に依存しない柔軟なセレクタ指定
* 会社名と時給を別々に取得し、データの整合性を確保
* 無効なデータの自動スキップ機能


5. データクレンジング


* 1000円未満の時給を除外
* 正規表現で数値のみを抽出
* 特殊なケース（1600円台）の標準化


6. データベース保存の安全性


* SQLパラメータ化でインジェクション対策
* トランザクション単位でのコミット
* 保存時の即時エラー検出


7. 進捗と結果の可視化


* リアルタイムの保存状況表示
* 区ごとの集計結果の自動計算
* 平均時給と総件数の表示


8. 拡張性への配慮


* 地区名をパラメータ化
* URL指定で異なる地域のデータ取得が可能
* テーブル構造で複数地区のデータを管理

#### 港区のスクレイピング

In [1]:
import re
import csv
import time
import requests
from bs4 import BeautifulSoup

def scrape_and_save_to_csv(base_url, csv_filename, max_pages=50):
    """
    base_url:
        ページ番号以外の共通部分。末尾に「?pageNo={page}」を付与して利用します。

    csv_filename:
        CSV出力のファイル名。

    max_pages:
        安全のための最大ページ数。デフォルトは50ですが、必要に応じて増減してください。
    """

    results = []

    # 正規表現パターン: "1600" の直後に4桁の数字が続くか → "1600" に置換
    pattern_remove_1600 = re.compile(r'1600\d{4}')

    for page_no in range(1, max_pages+1):
        # ページURL生成
        url = f"{base_url}?pageNo={page_no}"

        print(f"Fetching page: {url}")

        # リクエスト 前に3秒待機
        time.sleep(3)

        # ページ取得
        response = requests.get(url)
        # エラーなどでページが存在しない場合はそこで終了
        if response.status_code != 200:
            print(f"Page {page_no} not found (status: {response.status_code}). Stop.")
            break

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # カード単位で求人情報を取得（構造によって変更してください）
        # 例: "section.jobOfferCard" など、実際のHTMLを確認して適宜変更。
        # 以下は汎用的に "section" を全部取ってきて、その中に会社名・時給があるかを探す例。
        cards = soup.find_all("section")

        # フラグ: 1ページ分のcardが0件 または 有効なcardが何もない → 次のページ無さそうなら終了
        valid_card_found = False

        for card in cards:
            # 会社名を取得
            company_name_elem = card.select_one("div.shopNameWrap > h2")
            if not company_name_elem:
                continue
            company_name = company_name_elem.get_text(strip=True)

            # 時給を取得
            wage_elem = card.select_one("li.baseInformationSet.wage > div.baseInformationFirstContent")
            if not wage_elem:
                continue

            wage_text_original = wage_elem.get_text(strip=True)

            # "1600" のあとに4桁の数字 → "1600" に置換
            wage_text_fixed = re.sub(pattern_remove_1600, '1600', wage_text_original)

            # 数値抽出
            # 例: "時給1600～2000円" → ['1600','2000']
            # 最初の数字を最低時給とする
            nums = re.findall(r'\d+', wage_text_fixed)
            if not nums:
                continue

            min_wage = int(nums[0])  # 最初の数字を最低時給とみなす

            # 3桁以下(例: 100 ~ 999以下)は無視する
            if min_wage < 1000:
                # 例: 900とか700は日当表記などの可能性があるのでスキップ
                continue

            # ここまでこれたら有効データ
            valid_card_found = True

            results.append({
                "company_name": company_name,
                "wage_info": min_wage
            })

        # 有効なカードが0件の場合は「次のページはない」と判断して終了してもよい
        # ただし、中には他のページがまだある場合もあるので、判断が難しい場合は
        # ここでは card が見つからなくても break はしないことにします。
        # もし、次ページでは求人が本当にない、かつ 404 にもならないケースがあれば
        # 追加で条件を入れる必要があります。
        if not valid_card_found:
            print(f"No valid job found on page {page_no}. Stop.")
            break

    # すべてのページを取得後、CSV出力
    with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["company_name", "wage_info"])
        writer.writeheader()
        writer.writerows(results)

    # コンソールにも出力確認
    for row in results:
        print(f"{row['company_name']},{row['wage_info']}")


if __name__ == "__main__":
    # base_url は「?pageNo=」より前の部分
    # 例: "https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/"
    base_url = "https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/"
    csv_filename = "wage_info.csv"

    scrape_and_save_to_csv(base_url, csv_filename)
    print("==== CSV出力が完了しました ====")

Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=1
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=2
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=3
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=4
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=5
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=6
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=7
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=8
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=9
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=10
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=11
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=12
Fetching page: https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/?pageNo=13
Fetching page: https://baito.mynav

#### 足立区のスクレイピング

In [2]:
import re
import csv
import time
import requests
from bs4 import BeautifulSoup

def scrape_and_save_to_csv(base_url, csv_filename, max_pages=50):
    """
    base_url:
        ページ番号以外の共通部分。末尾に「?pageNo={page}」を付与して利用します。

    csv_filename:
        CSV出力のファイル名。

    max_pages:
        安全のための最大ページ数。デフォルトは50ですが、必要に応じて増減してください。
    """

    results = []

    # 正規表現パターン: "1600" の直後に4桁の数字が続くか → "1600" に置換
    pattern_remove_1600 = re.compile(r'1600\d{4}')

    for page_no in range(1, max_pages+1):
        # ページURL生成
        url = f"{base_url}?pageNo={page_no}"

        print(f"Fetching page: {url}")

        # リクエスト 前に3秒待機
        time.sleep(3)

        # ページ取得
        response = requests.get(url)
        # エラーなどでページが存在しない場合はそこで終了
        if response.status_code != 200:
            print(f"Page {page_no} not found (status: {response.status_code}). Stop.")
            break

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # カード単位で求人情報を取得（構造によって変更してください）
        # 例: "section.jobOfferCard" など、実際のHTMLを確認して適宜変更。
        # 以下は汎用的に "section" を全部取ってきて、その中に会社名・時給があるかを探す例。
        cards = soup.find_all("section")

        # フラグ: 1ページ分のcardが0件 または 有効なcardが何もない → 次のページ無さそうなら終了
        valid_card_found = False

        for card in cards:
            # 会社名を取得
            company_name_elem = card.select_one("div.shopNameWrap > h2")
            if not company_name_elem:
                continue
            company_name = company_name_elem.get_text(strip=True)

            # 時給を取得
            wage_elem = card.select_one("li.baseInformationSet.wage > div.baseInformationFirstContent")
            if not wage_elem:
                continue

            wage_text_original = wage_elem.get_text(strip=True)

            # "1600" のあとに4桁の数字 → "1600" に置換
            wage_text_fixed = re.sub(pattern_remove_1600, '1600', wage_text_original)

            # 数値抽出
            # 例: "時給1600～2000円" → ['1600','2000']
            # 最初の数字を最低時給とする
            nums = re.findall(r'\d+', wage_text_fixed)
            if not nums:
                continue

            min_wage = int(nums[0])  # 最初の数字を最低時給とみなす

            # 3桁以下(例: 100 ~ 999以下)は無視する
            if min_wage < 1000:
                # 例: 900とか700は日当表記などの可能性があるのでスキップ
                continue

            # ここまでこれたら有効データ
            valid_card_found = True

            results.append({
                "company_name": company_name,
                "wage_info": min_wage
            })

        # 有効なカードが0件の場合は「次のページはない」と判断して終了してもよい
        # ただし、中には他のページがまだある場合もあるので、判断が難しい場合は
        # ここでは card が見つからなくても break はしないことにします。
        # もし、次ページでは求人が本当にない、かつ 404 にもならないケースがあれば
        # 追加で条件を入れる必要があります。
        if not valid_card_found:
            print(f"No valid job found on page {page_no}. Stop.")
            break

    # すべてのページを取得後、CSV出力
    with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["company_name", "wage_info"])
        writer.writeheader()
        writer.writerows(results)

    # コンソールにも出力確認
    for row in results:
        print(f"{row['company_name']},{row['wage_info']}")


if __name__ == "__main__":
    # base_url は「?pageNo=」より前の部分
    # 例: "https://baito.mynavi.jp/tokyo/city-34/kd-11_3101/"
    base_url = "https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/"
    csv_filename = "wage_info.csv"

    scrape_and_save_to_csv(base_url, csv_filename)
    print("==== CSV出力が完了しました ====")

Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=1
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=2
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=3
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=4
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=5
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=6
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=7
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=8
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=9
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=10
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=11
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=12
Fetching page: https://baito.mynavi.jp/tokyo/city-52/kd-11_3101/?pageNo=13
Fetching page: https://baito.mynav